In [ ]:
import pandas as pd
import json
import os
import re
from openai import OpenAI #estamos la clase concreta OpenAI del módulo openai
from dotenv import load_dotenv #importamos una función concreta del módulo
load_dotenv("template.env")

In [ ]:
# Acceder a la clave de API de OpenAI
api_key = os.getenv("OPENAI_API_KEY")

# Asegurarte de que la clave de API se haya cargado correctamente
if api_key is None:
    raise ValueError('La clave de API no está configurada en el archivo .env')
    
client = OpenAI() #creando un objeto de la clase

In [ ]:
dataset_folder = os.getenv('DATASET_FOLDER')
dataset_path = str(dataset_folder) + "MMLU_pro.xlsx"

df = pd.read_excel(dataset_path)
#df = df.sample(30)
df.head()

In [ ]:
#FUNCTION DECLARATION
MODEL = "gpt-4o-mini"

#Generate description of multiple question
def generate_description(instr,op_a,op_b,op_c,op_d,op_e,op_f,op_g,op_h,op_i,op_j):
	description = json.dumps({
        "instruction": instr,
        "options": {
            "A": op_a,
            "B": op_b,
            "C": op_c,
            "D": op_d,
			"E": op_e,
            "F": op_f,
            "G": op_g,
            "H": op_h,
			"I": op_i,
            "J": op_j
        }
    })
	return description

#Generate one task
def generate_task(index,prompt,desc):
	task = {
        "custom_id": f"task-{index}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            # Esto es lo que tendrías en tu llamada a la API de Chat Completions
            "model": MODEL,
            "temperature": 0,
            "response_format": { 
                "type": "json_object"
            },
            "messages": [
                {
                    "role": "system",
                    "content": prompt
                },
                {
                    "role": "user",
                    "content": desc
                }
            ],
        }
    }
	return task

def extract_basic(new_line):
	new_line = new_line["body"]["messages"][1]["content"]
	return json.loads(new_line)

def create_file_from_tasks(tasks,file_name):
	with open(file_name, 'w') as file:
		for obj in tasks:
			file.write(json.dumps(obj) + '\n')

def create_batch(file_name):
	batch_file = client.files.create(
		file = open(file_name, "rb"),
		purpose = "batch"
	)
	batch_job = client.batches.create(
		input_file_id = batch_file.id,
		endpoint = "/v1/chat/completions",
		completion_window = "24h"
	)
	return batch_job

def clean_text(text):
    if isinstance(text, str):
        text = re.sub(r"[\x00-\x08\x0B\x0C\x0E-\x1F\x7F\t]", "", text)
        return text.strip()
    return text

def del_ch(string,array):
	string = str(string)
	for ch in array:
		string = string.replace(ch,"")
	return string

def df_array_divide(json_object,array_name):
	option_let = ["A","B","C","D","E","F","G","H","I","J","K","L"]
	keys = json_object.keys()
	res = "{"
	array = ""

	ch_del = ["'","\n","[","]","\\","\""]
	for i in range(0,len(keys)):
		if (keys[i] == array_name):
			array = json_object.values[i]
		else:
			value = del_ch(json_object.values[i],ch_del)
			res += f"\"{keys[i]}\":\"{value}\","

	ch_del2 = ["\n","[","]","\\"]
	array = del_ch(array,ch_del2)
	array = array.replace("\"","'")
	array = array.split("' '")

	for i in range(0,len(array)):
		array[i] = del_ch(array[i],ch_del)
	
	for i in range(0,10):
		try:
			res += f"\"{option_let[i]}\":\"{array[i]}\","
		except:
			res += f"\"{option_let[i]}\":\"not the answer\","
	
	res = res[:-1] +"}"
	res = clean_text(res)
	try:
		res = json.loads(res)
	except:
		print(res)
	return res

def create_task_from_json(json_object,index,prompt,rename_func=None):
	if (isinstance(json_object,str)):
		json_object = json.loads(json_object)
	if (rename_func == None):
		rename_func = ['instruction',
				 'option_a','option_b','option_c','option_d','option_e',
				 'option_f','option_g','option_h','option_i','option_j']
		
	try:
		instruction = json_object[rename_func[0]]
		option_a = json_object[rename_func[1]]
		option_b = json_object[rename_func[2]]
		option_c = json_object[rename_func[3]]
		option_d = json_object[rename_func[4]]
	except:
		print(json_object)
		print(index)

	try:
		option_e = json_object[rename_func[5]]
	except:
		option_e = "not the answer"
	try:
		option_f = json_object[rename_func[6]]
	except:
		option_f = "not the answer"
	try:
		option_g = json_object[rename_func[7]]
	except:
		option_g = "not the answer"
	try:
		option_h = json_object[rename_func[8]]
	except:
		option_h = "not the answer"
	try:
		option_i = json_object[rename_func[9]]
	except:
		option_i = "not the answer"
	try:
		option_j = json_object[rename_func[10]]
	except:
		option_j = "not the answer"

	description = generate_description(
		instruction, option_a, option_b, option_c, option_d, option_e, option_f, option_g, option_h,option_i, option_j,
	)
    
	task = generate_task(
		index, prompt, description,
	)
	return task

def create_task_array_from_dataframe(df,prompt,rename_func = None):
	tasks = []
	for index, row in df.iterrows():
		row = df_array_divide(row,"options")
		task = create_task_from_json(row,index,prompt,rename_func)
		tasks.append(task)
	return tasks

def extract_data_paraphrase(new_line):
	res = new_line["response"]["body"]["choices"][0]["message"]["content"]
	res = json.loads(res)
	return res

def get_line_file(file_name,line,extract_func):
	with open(file_name, 'r') as f:
		for line_number, theline in enumerate(f):
			if line_number == line:
				res = theline
				break
	res = json.loads(res)
	return extract_func(res)

def create_task_array_from_filename(file_name,prompt,extract_func,rename_func):
	tasks = []
	with open(file_name, 'r') as f:
		lines = len(f.readlines())
	for i in range(0,lines):
		try:
			line_new = get_line_file(file_name,i,extract_func)
		except:
			print(i)
		task = create_task_from_json(line_new,i,prompt,rename_func)
		tasks.append(task)
	
	return tasks

In [ ]:
#PROMPTS

#PARAPHRASING PROMPT
categorize_system_prompt_paraphrase ='''
Your goal is to paraphrase multiple choice questions from JSON objects. You will be provided with a JSON object containing a multiple choice question and you will output a JSON object with the reworded question.

For each row paraphrase the question while maintaining the original meaning and paraphrase the options as well. Ensure the paraphrase is at least 500 characters long between options and question.

Do not paraphrase options that are 'not the answer'

The output JSON objects should be in the following format:

{paraphrased_question: string, option_a: string, option_b: string, option_c: string, option_d: string, option_e: string, option_f: string, option_g: string, option_h: string, option_i: string, option_j: string,}

Ensure that the question is presented differently but conveys the same idea. Keep the JSON format in the output with '{' and '}'.
'''

#ANSWER PROMPT
categorize_system_prompt_answer = '''
Your goal is to evaluate multiple choice questions from JSON objects. You will be provided with a JSON object containing a multiple choice question and you will output a JSON object with the evaluation of the question.

For each row evaluate the question.

The output JSON objects should be in the following format:

{answer: string, // The selected option key for the question, limited to 'A', 'B', 'C', 'D' , 'E', 'F', 'G', 'H' , 'I' or 'J'}

Keep the JSON format in the answer with '{' and '}'.
'''

In [ ]:
#Set output folder
output_folder = os.getenv("OUTPUT_FOLDER")
def out_file(file_name): return (str(output_folder) + file_name)

In [ ]:
#PARAPHRASING TASK
r_func = ["question","A","B","C","D","E","F","G","H","I","J"]
tasks_array = [create_task_array_from_dataframe(df,categorize_system_prompt_paraphrase,r_func)]
file_array = [out_file("batch_job_mmlu_pro_paraphrase.jsonl")]

In [ ]:
#ANSWERING TASK
r_func = ["question","A","B","C","D","E","F","G","H","I","J"]
file_name = out_file("batch_job_mmlu_pro_paraphrase_result.jsonl")
rename_func = ['paraphrased_question','option_a','option_b','option_c','option_d','option_e','option_f','option_g','option_h','option_i','option_j']

tasks_array = [
	create_task_array_from_dataframe(df,categorize_system_prompt_answer,r_func),
	create_task_array_from_filename(file_name,categorize_system_prompt_answer,extract_data_paraphrase,rename_func)
	]
file_array = [out_file("batch_job_mmlu_pro_answer_original.jsonl"),out_file("batch_job_mmlu_pro_answer_paraphrase.jsonl")]

In [ ]:
#GENERATE TASK FILES
for i in range(0,len(tasks_array)):
	create_file_from_tasks(tasks_array[i],file_array[i])

In [ ]:
#GENERATE BATCH
batch_jobs = []
for i in range(0,len(tasks_array)):
	ba_jo= create_batch(file_array[i])
	batch_jobs.append(ba_jo)

In [ ]:
#COMPLETION_CHECK
for i in range(0,len(batch_jobs)):
	batch = batch_jobs[i]
	batch = client.batches.retrieve(batch.id)
	print(batch)
	result_file_id = batch.output_file_id
	print(batch.status)

In [ ]:
#OUTPUT FILES GENERATOR
for i in range(0,len(batch_jobs)):
	batch = batch_jobs[i]
	batch = client.batches.retrieve(batch.id)
	result_file_id = batch.output_file_id

	result = client.files.content(result_file_id).content

	result_file_name = file_array[i].replace(".json","_result.json")

	with open(result_file_name, 'wb') as file:
		file.write(result)

In [ ]:
#Cleaning function
def clean():
	with open(out_file("batch_job_mmlu_pro_paraphrase.jsonl"), 'r') as f:
		lines = len(f.readlines())
	
	rows = []
	stats = [0,0]

	f_o = out_file("batch_job_mmlu_pro_paraphrase.jsonl")
	f_ph = out_file("batch_job_mmlu_pro_answer_paraphrase.jsonl")
	f_a = out_file("batch_job_mmlu_pro_answer_original_result.jsonl")
	f_ph_a = out_file("batch_job_mmlu_pro_answer_paraphrase_result.jsonl")

	for i in range(0,lines):
		try:
			dt_o = get_line_file(f_o,i,extract_basic)
			dt_ph = get_line_file(f_ph,i,extract_basic)
			dt_a = get_line_file(f_a,i,extract_data_paraphrase)
			dt_ph_a = get_line_file(f_ph_a,i,extract_data_paraphrase)

			row = df.iloc[i]
			rows.append([
      	 	    dt_o['instruction'],
     	    	dt_o['options']["A"],
    	     	dt_o['options']["B"],
    	     	dt_o['options']["C"],
    	     	dt_o['options']["D"],
				dt_o['options']["E"],
   		      	dt_o['options']["F"],
   	  	    	dt_o['options']["G"],
        	 	dt_o['options']["H"],
				dt_o['options']["I"],
      	 	  	dt_o['options']["J"],
      		   	row['answer'],
				dt_a["answer"],
				clean_text(dt_ph['instruction']),
				clean_text(dt_ph['options']["A"]),
				clean_text(dt_ph['options']["B"]),
				clean_text(dt_ph['options']["C"]),
				clean_text(dt_ph['options']["D"]),
				clean_text(dt_ph['options']["E"]),
				clean_text(dt_ph['options']["F"]),
				clean_text(dt_ph['options']["G"]),
				clean_text(dt_ph['options']["H"]),
				clean_text(dt_ph['options']["I"]),
				clean_text(dt_ph['options']["J"]),
				dt_ph_a["answer"],
			])

			if(row['answer'] == dt_a["answer"]): stats[0]+=1
			if(row['answer'] == dt_ph_a["answer"]): stats[1]+=1
		except:
			print(i)
	
	stats[0] = stats[0]/lines
	stats[1] = stats[1]/lines
	
	stats = pd.DataFrame([{'acuracy': stats[0],'acuracy_ph': stats[1],}])
	return pd.DataFrame(rows),stats

file_name = out_file('FinalResults.xlsx')
clean_dtset,stat = clean()

In [ ]:
with pd.ExcelWriter(file_name) as writer:
	clean_dtset.to_excel(writer, sheet_name='Results',index=False)
	stat.to_excel(writer, sheet_name='Stats',engine='xlsxwriter',index=False)